L7 - L E A S I N G  


In [1]:
import re
from pathlib import Path

# ===== CONFIGURAÇÃO =====
pasta_entrada = Path(r"C:\projetos\itau_superveniencia\data\Telas Pretas")
pasta_output = Path(r"C:\projetos\itau_superveniencia\output\Telas Pretas")

# prefixo da subpasta para não misturar arquivos com mesmo nome
usar_prefixo_da_subpasta = False
# =======================

pasta_output.mkdir(parents=True, exist_ok=True)

padrao = re.compile(r"^@(\d+/\d+)[\s\S]*?^#\1\s*$", re.MULTILINE)

total_arquivos_txt = 0
total_contratos_novos = 0
total_ignorados_iguais = 0
total_duplicados_diferentes = 0

for arquivo_entrada in pasta_entrada.rglob("*.txt"):
    texto = arquivo_entrada.read_text(encoding="utf-8", errors="replace")

    # Pasta do tipo de arquivo (nome do TXT sem extensão)
    tipo_arquivo = arquivo_entrada.stem

    # prefixo da subpasta para não misturar arquivos com mesmo nome
    if usar_prefixo_da_subpasta:
        rel = arquivo_entrada.relative_to(pasta_entrada)
        prefixo = rel.parts[0] if len(rel.parts) >= 2 else "_RAIZ"
        pasta_tipo = pasta_output / f"{prefixo}_{tipo_arquivo}"
    else:
        pasta_tipo = pasta_output / tipo_arquivo

    pasta_tipo.mkdir(parents=True, exist_ok=True)

    contador_no_txt = 0

    for bloco in padrao.finditer(texto):
        contrato = bloco.group(1)              # ex: 8/0067078
        conteudo = bloco.group(0).strip() + "\n"

        nome_base = contrato.replace("/", "_")  # 8_0067078
        caminho_principal = pasta_tipo / f"{nome_base}.txt"

        # Caso 1: ainda não existe
        if not caminho_principal.exists():
            caminho_principal.write_text(conteudo, encoding="utf-8")
            total_contratos_novos += 1
            contador_no_txt += 1
            continue

        # Caso 2: existe -> compara
        conteudo_existente = caminho_principal.read_text(encoding="utf-8", errors="replace")

        if conteudo_existente == conteudo:
            # Mesmo contrato, mesma informação -> ignora
            total_ignorados_iguais += 1
            continue

        # Caso 3: mesmo contrato, conteúdo diferente -> salva DUP
        i = 1
        while True:
            caminho_dup = pasta_tipo / f"{nome_base}_DUP{i}.txt"
            if not caminho_dup.exists():
                caminho_dup.write_text(conteudo, encoding="utf-8")
                total_duplicados_diferentes += 1
                contador_no_txt += 1
                break
            i += 1

    print(f"{arquivo_entrada.name} -> {pasta_tipo.name}: {contador_no_txt} novos/duplicados (ignorados: dedup por igualdade)")
    total_arquivos_txt += 1

print("-" * 70)
print(f"Arquivos TXT processados       : {total_arquivos_txt}")
print(f"Contratos gravados (novos+dup) : {total_contratos_novos + total_duplicados_diferentes}")
print(f"Novos                          : {total_contratos_novos}")
print(f"Ignorados (iguais)             : {total_ignorados_iguais}")
print(f"Duplicados (conteúdo diferente): {total_duplicados_diferentes}")
print(f"Saída em                       : {pasta_output.resolve()}")


ComposicaoSaldoDevedor.txt -> ComposicaoSaldoDevedor: 22903 novos/duplicados (ignorados: dedup por igualdade)
ConsultaInicialdoContrato.txt -> ConsultaInicialdoContrato: 22903 novos/duplicados (ignorados: dedup por igualdade)
DadosCadastraisCliente.txt -> DadosCadastraisCliente: 22903 novos/duplicados (ignorados: dedup por igualdade)
DetalhamentoCadastroConsultaBens.txt -> DetalhamentoCadastroConsultaBens: 22903 novos/duplicados (ignorados: dedup por igualdade)
DetalhesContrato.txt -> DetalhesContrato: 22903 novos/duplicados (ignorados: dedup por igualdade)
----------------------------------------------------------------------
Arquivos TXT processados       : 5
Contratos gravados (novos+dup) : 114515
Novos                          : 114405
Ignorados (iguais)             : 0
Duplicados (conteúdo diferente): 110
Saída em                       : C:\projetos\itau_superveniencia\output\Telas Pretas


listagem


In [2]:
import re
from pathlib import Path

# ================= CONFIGURAÇÃO =================
arquivo_entrada = Path(
    r"C:\projetos\itau_superveniencia\data\listagem\ListagemCadastroConsultaBens.txt"
)

pasta_saida = Path(
    r"C:\projetos\itau_superveniencia\output\Telas Pretas\ListagemCadastroConsultaBens"
)
pasta_saida.mkdir(parents=True, exist_ok=True)
# ==============================================

# -------- REGEX (padrão correto: @X/NNNNNNN ... #X/NNNNNNN) --------
padrao_bloco = re.compile(
    r"^@(\d+/\d+)[\s\S]*?^#\1\s*$",
    re.MULTILINE
)

# -------- PROCESSAMENTO --------
if not arquivo_entrada.exists():
    raise FileNotFoundError(f"Arquivo de entrada não encontrado: {arquivo_entrada.resolve()}")

texto = arquivo_entrada.read_text(encoding="utf-8", errors="replace")

total = 0
for bloco in padrao_bloco.finditer(texto):
    chave = bloco.group(1)  # ex: 8/0067078
    conteudo = bloco.group(0).strip() + "\n"

    nome_arquivo = chave.replace("/", "_") + ".txt"
    (pasta_saida / nome_arquivo).write_text(conteudo, encoding="utf-8")
    total += 1

print("-" * 70)
print(f"Arquivo processado     : {arquivo_entrada.name}")
print(f"Contratos gerados      : {total}")
print(f"Saída em               : {pasta_saida.resolve()}")
print("-" * 70)


----------------------------------------------------------------------
Arquivo processado     : ListagemCadastroConsultaBens.txt
Contratos gerados      : 22903
Saída em               : C:\projetos\itau_superveniencia\output\Telas Pretas\ListagemCadastroConsultaBens
----------------------------------------------------------------------


Validation - gera um excel no final 

In [3]:
from pathlib import Path
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from openpyxl.utils import get_column_letter

# ===== CONFIGURAÇÃO =====
txt_contratos_esperados = Path(r"C:/projetos/itau_superveniencia/data/contratos2014.txt")

# Raiz onde estão as pastas por tipo (cada tipo contém vários .txt com nome do contrato)
raiz_gerados = Path(r"C:/projetos/itau_superveniencia/output/Telas Pretas")

# Pasta de saída (Excel + TXTs de faltantes)
pasta_saida = Path(r"C:/projetos/itau_superveniencia/output/validacao")
pasta_saida.mkdir(parents=True, exist_ok=True)

arquivo_excel = pasta_saida / "validacao_por_tipo.xlsx"

# TXTs de faltantes (um por tipo + geral)
pasta_txt_faltantes = pasta_saida / "faltantes_txt"
pasta_txt_faltantes.mkdir(parents=True, exist_ok=True)
txt_faltantes_geral = pasta_txt_faltantes / "faltantes_geral.txt"
# =======================


def sanitizar_nome_aba(nome: str) -> str:
    proibidos = set(r':\/?*[]')
    limpo = "".join("_" if c in proibidos else c for c in nome).strip()
    return limpo[:31] if limpo else "SEM_NOME"


def ajustar_larguras(ws):
    for col in range(1, ws.max_column + 1):
        max_len = 0
        for row in range(1, ws.max_row + 1):
            val = ws.cell(row=row, column=col).value
            if val is None:
                continue
            max_len = max(max_len, len(str(val)))
        ws.column_dimensions[get_column_letter(col)].width = min(max_len + 2, 80)


def somente_digitos(s: str) -> str:
    return "".join(ch for ch in s if ch.isdigit())


def normalizar_esperado_para_int(linha: str):
    d = somente_digitos(linha.strip())
    if not d:
        return None
    return int(d)


def extrair_numero_contrato_do_arquivo(stem: str):
    # Ex: "8_5066665" -> 5066665
    if "_" in stem:
        _, sufixo = stem.split("_", 1)
    else:
        sufixo = stem
    sufixo = somente_digitos(sufixo)
    if not sufixo:
        return None
    return int(sufixo)


def salvar_txt_lista(caminho: Path, numeros: list[int]):
    caminho.write_text("\n".join(str(n) for n in numeros) + ("\n" if numeros else ""), encoding="utf-8")


# ---- Checks de existência ----
if not txt_contratos_esperados.exists():
    raise FileNotFoundError(f"Arquivo de contratos esperados não encontrado: {txt_contratos_esperados.resolve()}")

if not raiz_gerados.exists():
    raise FileNotFoundError(f"Pasta raiz dos gerados não encontrada: {raiz_gerados.resolve()}")

# ---- Lê contratos esperados (preciso) ----
contratos_esperados = set()
for linha in txt_contratos_esperados.read_text(encoding="utf-8", errors="replace").splitlines():
    if not linha.strip():
        continue
    n = normalizar_esperado_para_int(linha)
    if n is not None:
        contratos_esperados.add(n)

if not contratos_esperados:
    raise ValueError("Nenhum contrato válido foi lido do arquivo de contratos esperados.")

# ---- Pastas tipo ----
pastas_tipo = sorted({p.parent for p in raiz_gerados.rglob("*.txt")})
if not pastas_tipo:
    raise ValueError(f"Nenhum .txt encontrado dentro de: {raiz_gerados.resolve()}")

# ---- Calcula resultado por tipo ----
resultado_por_tipo = []
faltantes_geral_set = set()

for pasta in pastas_tipo:
    tipo = pasta.name

    # encontrado: numero -> [arquivos]
    encontrados_map = {}  # int -> list[str]
    invalidos = 0

    for arq in pasta.glob("*.txt"):
        n = extrair_numero_contrato_do_arquivo(arq.stem)
        if n is None:
            invalidos += 1
            continue
        encontrados_map.setdefault(n, []).append(arq.name)

    encontrados_set = set(encontrados_map.keys())

    faltantes = sorted(contratos_esperados - encontrados_set)
    extras = sorted(encontrados_set - contratos_esperados)

    faltantes_geral_set.update(faltantes)

    # lista detalhada de extras com nome(s) de arquivo
    extras_detalhado = []
    for n in extras:
        for fname in sorted(encontrados_map.get(n, [])):
            extras_detalhado.append((n, fname))

    resultado_por_tipo.append({
        "tipo": tipo,
        "pasta": pasta,
        "esperado": len(contratos_esperados),
        "encontrado": len(encontrados_set),
        "faltantes": faltantes,
        "extras": extras,
        "extras_detalhado": extras_detalhado,  # (numero, arquivo)
        "invalidos": invalidos,
    })

# ---- TXT faltantes por tipo + geral ----
for r in resultado_por_tipo:
    salvar_txt_lista(pasta_txt_faltantes / f"faltantes_{r['tipo']}.txt", r["faltantes"])

salvar_txt_lista(txt_faltantes_geral, sorted(faltantes_geral_set))

# ---- Excel ----
wb = Workbook()

# Resumo
ws_resumo = wb.active
ws_resumo.title = "Resumo"
ws_resumo.append(["Tipo", "Esperado", "Encontrado", "Faltantes", "Extras", "Inválidos", "Pasta"])
for cell in ws_resumo[1]:
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

for r in resultado_por_tipo:
    ws_resumo.append([
        r["tipo"],
        r["esperado"],
        r["encontrado"],
        len(r["faltantes"]),
        len(r["extras"]),
        r["invalidos"],
        str(r["pasta"].resolve()),
    ])

ws_resumo.freeze_panes = "A2"
ajustar_larguras(ws_resumo)

# Abas por tipo
nomes_usados = set()
for r in resultado_por_tipo:
    base = sanitizar_nome_aba(r["tipo"])
    nome_aba = base
    i = 2
    while nome_aba in nomes_usados:
        sufixo = f"_{i}"
        nome_aba = (base[:31 - len(sufixo)] + sufixo)[:31]
        i += 1
    nomes_usados.add(nome_aba)

    ws = wb.create_sheet(title=nome_aba)

    # Cabeçalho
    ws.append(["Tipo", r["tipo"]])
    ws.append(["Pasta", str(r["pasta"].resolve())])
    ws.append(["Total esperado", r["esperado"]])
    ws.append(["Total encontrado", r["encontrado"]])
    ws.append(["Qtd faltantes", len(r["faltantes"])])
    ws.append(["Qtd extras", len(r["extras"])])
    ws.append(["Arquivos inválidos (nome não parseável)", r["invalidos"]])
    ws.append([])

    for row_cells in ws["A1":"A7"]:
        row_cells[0].font = Font(bold=True)

    # Faltantes
    ws.append(["FALTANTES (contratos esperados que NÃO foram encontrados nesta pasta)"])
    ws["A9"].font = Font(bold=True)
    ws.append(["Contrato (numérico)"])
    ws["A10"].font = Font(bold=True)

    row = 11
    if r["faltantes"]:
        for n in r["faltantes"]:
            ws.cell(row=row, column=1, value=n)
            row += 1
    else:
        ws.cell(row=row, column=1, value="(nenhum)")
        row += 1

    ws.append([])
    row += 1

    # Extras detalhados (com arquivos)
    ws.append(["EXTRAS (arquivos encontrados que NÃO pertencem ao lote esperado)"])
    ws.cell(row=row, column=1).font = Font(bold=True)
    row += 1

    ws.append(["Contrato extra (numérico)", "Arquivo (.txt)"])
    ws.cell(row=row, column=1).font = Font(bold=True)
    ws.cell(row=row, column=2).font = Font(bold=True)
    row += 1

    if r["extras_detalhado"]:
        for n, fname in r["extras_detalhado"]:
            ws.cell(row=row, column=1, value=n)
            ws.cell(row=row, column=2, value=fname)
            row += 1
    else:
        ws.cell(row=row, column=1, value="(nenhum)")

    ws.freeze_panes = "A11"
    ajustar_larguras(ws)

wb.save(arquivo_excel)

print(f"Excel gerado em: {arquivo_excel.resolve()}")
print(f"TXTs de faltantes por tipo em: {pasta_txt_faltantes.resolve()}")
print(f"TXT de faltantes geral em: {txt_faltantes_geral.resolve()}")


Excel gerado em: C:\projetos\itau_superveniencia\output\validacao\validacao_por_tipo.xlsx
TXTs de faltantes por tipo em: C:\projetos\itau_superveniencia\output\validacao\faltantes_txt
TXT de faltantes geral em: C:\projetos\itau_superveniencia\output\validacao\faltantes_txt\faltantes_geral.txt


validação + movimentação

In [ ]:
from pathlib import Path
import shutil
from openpyxl import load_workbook

# ================= CONFIGURAÇÃO =================
arquivo_excel_validacao = Path(
    r"C:\projetos\itau_superveniencia\output\validacao\L7 - L E A S I N G\validacao_por_tipo.xlsx"
)

raiz_output_telaspretas = Path(r"C:\projetos\itau_superveniencia\output\Telas Pretas")

# Pasta destino para extras
pasta_extras_raiz = raiz_output_telaspretas / "extras"

MOVER = True
# ==============================================

if not arquivo_excel_validacao.exists():
    raise FileNotFoundError(f"Excel de validação não encontrado: {arquivo_excel_validacao.resolve()}")

if not raiz_output_telaspretas.exists():
    raise FileNotFoundError(f"Raiz do output não encontrada: {raiz_output_telaspretas.resolve()}")

pasta_extras_raiz.mkdir(parents=True, exist_ok=True)

wb = load_workbook(arquivo_excel_validacao, data_only=True)

total_movidos = 0
total_nao_encontrados = 0
total_abas_processadas = 0

def achar_linha_extras(ws) -> int | None:
    """
    Encontra a linha onde começa o bloco:
    'EXTRAS (arquivos encontrados que NÃO pertencem ao lote esperado)'
    Retorna o número da linha, ou None.
    """
    for r in range(1, ws.max_row + 1):
        v = ws.cell(row=r, column=1).value
        if isinstance(v, str) and v.strip().startswith("EXTRAS"):
            return r
    return None

def achar_pasta_origem(ws) -> Path | None:
    """
    No topo da aba existe:
      A2 = 'Pasta'
      B2 = 'C:\\...\\output\\Telas Pretas\\<Tipo>'
    Retorna Path(B2).
    """
    for r in range(1, 15):  # topo
        a = ws.cell(row=r, column=1).value
        if isinstance(a, str) and a.strip().lower() == "pasta":
            b = ws.cell(row=r, column=2).value
            if isinstance(b, str) and b.strip():
                return Path(b.strip())
    return None

def achar_tipo(ws) -> str | None:
    """
    No topo:
      A1 = 'Tipo'
      B1 = '<Tipo>'
    """
    for r in range(1, 15):
        a = ws.cell(row=r, column=1).value
        if isinstance(a, str) and a.strip().lower() == "tipo":
            b = ws.cell(row=r, column=2).value
            if isinstance(b, str) and b.strip():
                return b.strip()
    return None

# Processa todas as abas, exceto Resumo
for nome_aba in wb.sheetnames:
    if nome_aba.strip().lower() == "resumo":
        continue

    ws = wb[nome_aba]

    pasta_origem = achar_pasta_origem(ws)
    tipo = achar_tipo(ws)

    if pasta_origem is None or tipo is None:
        # Aba fora do formato esperado
        continue

    linha_extras = achar_linha_extras(ws)
    if linha_extras is None:
        continue

    row = linha_extras + 2

    # Pasta destino do tipo
    destino_tipo = pasta_extras_raiz / tipo
    destino_tipo.mkdir(parents=True, exist_ok=True)

    movidos_nesta_aba = 0

    while row <= ws.max_row:
        arquivo_txt = ws.cell(row=row, column=2).value  # coluna B: "Arquivo (.txt)"
        if arquivo_txt is None:
            row += 1
            continue

        # Quando acaba a lista, normalmente vira "(nenhum)" ou entra em branco
        if isinstance(arquivo_txt, str):
            arquivo_txt = arquivo_txt.strip()
            if not arquivo_txt or arquivo_txt.startswith("("):
                break
        else:
            # Se não for string, não faz sentido aqui
            break

        origem = pasta_origem / arquivo_txt
        destino = destino_tipo / arquivo_txt

        if origem.exists():
            if MOVER:
                shutil.move(str(origem), str(destino))
            else:
                shutil.copy2(str(origem), str(destino))
            total_movidos += 1
            movidos_nesta_aba += 1
        else:
            total_nao_encontrados += 1

        row += 1

    print(f"Aba {nome_aba} (Tipo={tipo}): movidos {movidos_nesta_aba}")
    total_abas_processadas += 1

print("-" * 70)
print(f"Abas processadas           : {total_abas_processadas}")
print(f"Arquivos extras movidos    : {total_movidos}")
print(f"Extras não encontrados     : {total_nao_encontrados}")
print(f"Destino extras             : {pasta_extras_raiz.resolve()}")
print("-" * 70)


In [1]:
pip install reportlab 

  Using cached reportlab-4.4.7-py3-none-any.whl.metadata (1.7 kB)
  Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl.metadata (38 kB)
Using cached reportlab-4.4.7-py3-none-any.whl (2.0 MB)
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   -------------------- ------------------- 3.7/7.0 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 23.9 MB/s eta 0:00:00
Using cached charset_normalizer-3.4.4-cp312-cp312-win_amd64.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from pathlib import Path
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_LEFT


def txt_para_pdf(caminho_txt: Path, caminho_pdf: Path) -> None:
    texto = caminho_txt.read_text(encoding="utf-8", errors="ignore")

    doc = SimpleDocTemplate(
        str(caminho_pdf),
        pagesize=A4,
        leftMargin=2 * cm,
        rightMargin=2 * cm,
        topMargin=2 * cm,
        bottomMargin=2 * cm,
    )

    styles = getSampleStyleSheet()
    estilo = ParagraphStyle(
        name="Texto",
        parent=styles["Normal"],
        alignment=TA_LEFT,
        fontSize=10,
        leading=14,
    )

    story = []
    for linha in texto.splitlines():
        linha = (
            linha.replace("&", "&amp;")
                 .replace("<", "&lt;")
                 .replace(">", "&gt;")
        )
        if linha.strip() == "":
            story.append(Spacer(1, 6))
        else:
            story.append(Paragraph(linha, estilo))

    doc.build(story)


def pasta_txt_para_pdfs(pasta_base: str) -> None:
    pasta_base = Path(pasta_base)

    print(f"Pasta base: {pasta_base.resolve()}")
    if not pasta_base.exists():
        print("ERRO: a pasta base não existe.")
        return

    arquivos_txt = list(pasta_base.rglob("*.txt"))
    print(f"TXT encontrados: {len(arquivos_txt)}")

    if not arquivos_txt:
        print("ERRO: nenhum .txt encontrado. Verifique o caminho e se os arquivos terminam com .txt.")
        return

    gerados = 0
    for caminho_txt in arquivos_txt:
        try:
            pasta_tipo = caminho_txt.parent
            pasta_pdf = pasta_tipo / "pdf"
            pasta_pdf.mkdir(parents=True, exist_ok=True)

            caminho_pdf = pasta_pdf / f"{caminho_txt.stem}.pdf"
            txt_para_pdf(caminho_txt, caminho_pdf)

            gerados += 1
            print(f"OK: {caminho_txt.name} -> {caminho_pdf}")
        except Exception as e:
            print(f"FALHA: {caminho_txt} | {type(e).__name__}: {e}")

    print(f"Finalizado. PDFs gerados: {gerados}")


# Use caminho ABSOLUTO para evitar erro por caminho relativo
PASTA_BASE = r"C:\projetos\itau_superveniencia\output\Telas Pretas"
pasta_txt_para_pdfs(PASTA_BASE)


Pasta base: C:\projetos\itau_superveniencia\output\Telas Pretas
TXT encontrados: 137396
OK: 1_0006306.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0006306.pdf
OK: 1_0014227.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0014227.pdf
OK: 1_0035923.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0035923.pdf
OK: 1_0074509.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0074509.pdf
OK: 1_0097740.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0097740.pdf
OK: 1_0107213.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0107213.pdf
OK: 1_0146540.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0146540.pdf
OK: 1_0150644.txt -> C:\projetos\itau_superveniencia\output\Telas Pretas\ComposicaoSaldoDevedor\pdf\1_0150644.pdf


KeyboardInterrupt: 